In [2]:
# %%
import os
import dotenv
dotenv.load_dotenv()

# %%
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.globals import set_debug, set_verbose

set_verbose(True)
set_debug(True)

# Загрузка эмбеддингов и векторной БД
embedding = OpenAIEmbeddings()
vectorstore = FAISS.load_local("vuln_index", embedding, allow_dangerous_deserialization=True)

# Инициализация модели
model = ChatOpenAI(model_name="gpt-4o-mini")

# %%
# Кастомный промпт для кода
prompt = PromptTemplate.from_template("""

""")


qa_template = '''
You are a smart contract security expert.

Here is relevant context from known vulnerabilities:
{context}

Now analyze the following Solidity code:
{question}

Return a list of any detected vulnerabilities in this format:
[
  {{
    "line": "line number",
    "token": "problematic code",
    "problem": "short name",
    "severity": "low | medium | high",
    "explanation": "why it's dangerous",
    "migration": "how to fix"
  }}
]
'''
prompt = PromptTemplate(template=qa_template, input_variables=['context', 'question'])


# Создаем chain с retrieval
retriever = vectorstore.as_retriever(search_type="similarity", k=4)

qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={'prompt': prompt}
)

# %%
# Читаем код из snippets/
snippets_path = "snippets"
files = os.listdir(snippets_path)
code_snippets = []
for file in files:
    with open(os.path.join(snippets_path, file), "r") as f:
        code_snippets.append(f.read())

# %%
# Анализ одного сниппета
code = code_snippets[2]

# %%
# Исправленный вызов chain.invoke() без дублирующихся ключевых аргументов
result = qa_chain.invoke({"query": code})

# %%
print(code)
print(result)


[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0;\n\ncontract PiggyBridge is ReentrancyGuard {\n    mapping(address => uint256) public balances;\n    mapping(bytes32 => bool) public executedTxs;\n\n    function deposit() external payable {\n        balances[msg.sender] += msg.value;\n    }\n\n    function withdraw(uint256 amount, uint256 nonce, bytes memory signature) external nonReentrant {\n        require(balances[msg.sender] >= amount, \"Insufficient balance\");\n\n        bytes32 messageHash = keccak256(abi.encodePacked(msg.sender, amount, nonce));\n        require(!executedTxs[messageHash], \"Already executed\");\n\n        address signer = recoverSigner(messageHash, signature);\n        require(signer == msg.sender, \"Invalid signature\");\n\n        executedTxs[messageHash] = true;\n        balances[msg.sender] -= amount;\n        payable(msg.sender).transfer(amount);\n    }\n\n    function re

In [4]:
result

{'query': '// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0;\n\ncontract PiggyBridge is ReentrancyGuard {\n    mapping(address => uint256) public balances;\n    mapping(bytes32 => bool) public executedTxs;\n\n    function deposit() external payable {\n        balances[msg.sender] += msg.value;\n    }\n\n    function withdraw(uint256 amount, uint256 nonce, bytes memory signature) external nonReentrant {\n        require(balances[msg.sender] >= amount, "Insufficient balance");\n\n        bytes32 messageHash = keccak256(abi.encodePacked(msg.sender, amount, nonce));\n        require(!executedTxs[messageHash], "Already executed");\n\n        address signer = recoverSigner(messageHash, signature);\n        require(signer == msg.sender, "Invalid signature");\n\n        executedTxs[messageHash] = true;\n        balances[msg.sender] -= amount;\n        payable(msg.sender).transfer(amount);\n    }\n\n    function recoverSigner(bytes32 hash, bytes memory signature) public pure returns (add